In [5]:
import pyvisa

In [6]:
rm = pyvisa.ResourceManager('@py')
print(rm)
rm.list_resources()

Resource Manager of Visa Library at py


('ASRL/dev/ttyS31::INSTR',
 'ASRL/dev/ttyS30::INSTR',
 'ASRL/dev/ttyS29::INSTR',
 'ASRL/dev/ttyS28::INSTR',
 'ASRL/dev/ttyS27::INSTR',
 'ASRL/dev/ttyS26::INSTR',
 'ASRL/dev/ttyS25::INSTR',
 'ASRL/dev/ttyS24::INSTR',
 'ASRL/dev/ttyS23::INSTR',
 'ASRL/dev/ttyS22::INSTR',
 'ASRL/dev/ttyS21::INSTR',
 'ASRL/dev/ttyS20::INSTR',
 'ASRL/dev/ttyS19::INSTR',
 'ASRL/dev/ttyS18::INSTR',
 'ASRL/dev/ttyS17::INSTR',
 'ASRL/dev/ttyS16::INSTR',
 'ASRL/dev/ttyS15::INSTR',
 'ASRL/dev/ttyS14::INSTR',
 'ASRL/dev/ttyS13::INSTR',
 'ASRL/dev/ttyS12::INSTR',
 'ASRL/dev/ttyS11::INSTR',
 'ASRL/dev/ttyS10::INSTR',
 'ASRL/dev/ttyS9::INSTR',
 'ASRL/dev/ttyS8::INSTR',
 'ASRL/dev/ttyS7::INSTR',
 'ASRL/dev/ttyS6::INSTR',
 'ASRL/dev/ttyS5::INSTR',
 'ASRL/dev/ttyS4::INSTR',
 'ASRL/dev/ttyS3::INSTR',
 'ASRL/dev/ttyS2::INSTR',
 'ASRL/dev/ttyS1::INSTR',
 'ASRL/dev/ttyS0::INSTR',
 'ASRL/dev/ttyACM0::INSTR')

In [26]:
inst = rm.open_resource('ASRL/dev/ttyACM1::INSTR')

In [22]:
print(inst)

SerialInstrument at ASRL/dev/ttyACM1::INSTR


In [23]:
inst.write('three\n')

8

In [24]:
print(inst.read())

hey



In [25]:
print(inst.query('four\n'))

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [30]:
inst.write('one\n')

6

In [31]:
data = inst.read_raw()

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [28]:
#values = inst.query_binary_values('CURV?', datatype='d', is_big_endian=True)
values = inst.read_binary_values(datatype='H', is_big_endian=False, data_points=2, expect_termination=False)

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.